In [180]:
import numpy as np
import pandas as pd

import spotipy
import spotipy.oauth2 as oauth2
import datetime as dt
from tqdm.notebook import tqdm

import time
import Levenshtein

import matplotlib.pyplot as plt
%matplotlib inline

In [190]:
client_id='503ca6159ce0418fb4c366f5b0e0d0e0'
client_secret='6c659793ab9e46428d38da5a6f9ec526'
credentials = oauth2.SpotifyClientCredentials(
        client_id=client_id,
        client_secret=client_secret)

token = credentials.get_access_token()
spotify = spotipy.Spotify(auth=token)

/Users/shaunmbarry/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.
  import sys


In [5]:
df = pd.read_csv('no_need_to_check-38301.csv')

In [7]:
df = df[df.columns[1:]]

In [19]:
i = 0
pred_url = df['Predicted URL'][i]
track = spotify.track(pred_url)
album_types = []

album_types += [track['album']['album_type']]


'single'

## Add new column for charts in CSV file

In [26]:
df_seperate = pd.read_csv('./CombinedChartsUnique_AllBillboard.csv')

In [31]:
df_sep_debut = df_seperate[df_seperate.columns[4::7].tolist()].copy()
df_sep_debut 

,DebutChartDate_Christian,DebutChartDate_Country,DebutChartDate_DanceElectronic,DebutChartDate_Hot100,DebutChartDate_Latin,DebutChartDate_RAB,DebutChartDate_Rap,DebutChartDate_Rock
0,NaN,NaN,NaN,6/24/17,NaN,6/24/17,6/24/17,NaN
1,NaN,NaN,NaN,2/18/12,NaN,11/12/11,NaN,NaN
2,NaN,NaN,NaN,NaN,4/24/04,NaN,NaN,NaN
3,NaN,NaN,NaN,7/29/17,NaN,7/29/17,7/29/17,NaN
4,NaN,NaN,NaN,9/29/79,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
58472,NaN,NaN,NaN,NaN,1/22/11,NaN,NaN,NaN
58473,NaN,NaN,NaN,1/29/94,NaN,NaN,2/5/94,NaN
58474,NaN,7/4/98,NaN,NaN,NaN,NaN,NaN,NaN
58475,NaN,NaN,NaN,12/11/04,NaN,NaN,NaN,NaN


In [68]:
debut_names = np.asarray([string.split('_')[-1] for string in df_sep_debut.columns.tolist()])
debut_names

array(['Christian', 'Country', 'DanceElectronic', 'Hot100', 'Latin',
       'RAB', 'Rap', 'Rock'], dtype='<U15')

In [71]:
charts_appeared_on = []
for i in range(df_seperate.shape[0]):
    charts_appeared_on += [', '.join(debut_names[np.where(~np.isnan(df_seperate[i:(i+1)][df_seperate.columns[5::7].tolist()].values[0]))[0]])]

In [76]:
charts_df = pd.DataFrame(columns=['Chart Appearances'], data=charts_appeared_on)
new_df = pd.concat([df_seperate, charts_df], axis=1, sort=False)
new_df.to_csv('./CombinedChartsUnique_AllBillboard_chartsinfo.csv')

In [77]:
new_df

,Title,Artist,AlreadyBoilsofted,Problem,DebutChartDate_Christian,DebutChartPosition_Christian,NumWeeksOnChart_Christian,AvgChartPosition_Christian,PeakChartPosition_Christian,EndChartDate_Christian,...,EndChartDate_Rap,EndChartPosition_Rap,DebutChartDate_Rock,DebutChartPosition_Rock,NumWeeksOnChart_Rock,AvgChartPosition_Rock,PeakChartPosition_Rock,EndChartDate_Rock,EndChartPosition_Rock,Chart Appearances
0,4:00 AM,2 Chainz Featuring Travis Scott,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7/22/17,18.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Hot100, RAB, Rap"
1,4:00 AM,Melanie Fiona,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Hot100, RAB"
2,4:30 AM,Obie Bermudez,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Latin
3,4:44,JAY-Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8/5/17,19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Hot100, RAB, Rap"
4,5:15,The Who,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hot100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58472,Zun Zun Rompiendo Caderas,Wisin & Yandel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Latin
58473,Zunga Zeng,K7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2/5/94,23.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Hot100, Rap"
58474,TRUE,George Strait,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Country
58475,TRUE,Ryan Cabrera,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hot100


## Merge with new data

In [124]:
with open('all_urls_y2k+.txt', 'r+') as f:
    urls = np.asarray(f.read().split('\n'))

In [129]:
new_chart_row = df[df['Predicted URL'] == pred_url].copy()
assert new_chart_row.shape[0] == 1
new_chart_row

,Chart Artist Name,Spotify Artist Name,Chart Song Name,Spotify Song Name,Lev Dist,Predicted URL,Debut Year,Debut Date
1186,Glenn Frey,Glenn Frey,Sexy Girl,Sexy Girl,0.0,https://open.spotify.com/track/6c3rTjGR3FuGfTG...,1984,6/30/84


In [131]:
[charts_appeared_on[-1], 
              artist_genres_list[-1], 
              album_genres_list[-1],
              album_types[-1],
              album_names[-1]]

['Hot100',
 'classic country pop || oklahoma country',
 nan,
 'album',
 'The Allnighter']

In [138]:
new_cols = ['Chart Appearances', 'Sp Artist Genres','Sp Album Genres', 'Sp Album Type', 'Sp Album Name']
new_df_cols = pd.DataFrame([[charts_appeared_on[-1], 
              artist_genres_list[-1], 
              album_genres_list[-1],
              album_types[-1],
              album_names[-1]]], columns=new_cols)
new_df_cols

,Chart Appearances,Sp Artist Genres,Sp Album Genres,Sp Album Type,Sp Album Name
0,Hot100,classic country pop || oklahoma country,NaN,album,The Allnighter


In [149]:
new_chart_row = new_chart_row.reset_index(drop=True)

In [152]:
new_full_row = pd.concat([new_chart_row, new_df_cols], sort=False, axis=1)
new_full_row

,Chart Artist Name,Spotify Artist Name,Chart Song Name,Spotify Song Name,Lev Dist,Predicted URL,Debut Year,Debut Date,Chart Appearances,Sp Artist Genres,Sp Album Genres,Sp Album Type,Sp Album Name
0,Glenn Frey,Glenn Frey,Sexy Girl,Sexy Girl,0.0,https://open.spotify.com/track/6c3rTjGR3FuGfTG...,1984,6/30/84,Hot100,classic country pop || oklahoma country,NaN,album,The Allnighter


In [155]:
new_full_row_columns = ['Chart Artist Name',
 'Spotify Artist Name',
 'Chart Song Name',
 'Spotify Song Name',
 'Lev Dist',
 'Predicted URL',
 'Debut Year',
 'Debut Date',
 'Chart Appearances',
 'Sp Artist Genres',
 'Sp Album Genres',
 'Sp Album Type',
 'Sp Album Name']

In [191]:
final_df = pd.DataFrame([[None]*13], columns=new_full_row_columns)

charts_appeared_on = []
artist_genres_list = []
album_genres_list = []
album_types = []
album_names = []

for i, pred_url in tqdm(enumerate(urls)):
    # get charts appeared on
    
    new_chart_row = df[df['Predicted URL'] == pred_url].copy().reset_index(drop=True)
    chart_artist, chart_title = new_chart_row['Chart Artist Name'].values[0], new_chart_row['Chart Song Name'].values[0]
    sub_df = new_df[np.logical_and(new_df['Artist'] == chart_artist, new_df['Title'] == chart_title)]
    charts_appeared_on += [sub_df['Chart Appearances'].values[0]]
    
    # get spotify genres
    
    try:
        track = spotify.track(pred_url)
    except:
        token = credentials.get_access_token()
        spotify = spotipy.Spotify(auth=token)
        print('Bad url:', pred_url)
        track = spotify.track(pred_url)
        
    
    album_types += [track['album']['album_type']]
    album_names += [track['album']['name']]
    
    art_genres = []
    for art in track['artists']:
        try:
            artist = spotify.artist(art['id'])
        except:
            token = credentials.get_access_token()
            spotify = spotipy.Spotify(auth=token)
            artist = spotify.artist(art['id'])
        
        art_genres += artist['genres']

    try:
        album = spotify.album(track['album']['id'])
    except:
        token = credentials.get_access_token()
        spotify = spotipy.Spotify(auth=token)
        album = spotify.album(track['album']['id'])
    
    album_genres = album['genres']

    if art_genres == []:
        art_genres = np.nan
    else:
        art_genres = ' || '.join(art_genres)
    
    if album_genres == []:
        album_genres = np.nan
    else:
        album_genres = ' || '.join(album_genres)
        
    artist_genres_list += [art_genres]
    album_genres_list += [album_genres]
    
    new_df_cols = pd.DataFrame([[charts_appeared_on[-1], 
              artist_genres_list[-1], 
              album_genres_list[-1],
              album_types[-1],
              album_names[-1]]], columns=new_cols)
    new_full_row = pd.concat([new_chart_row, new_df_cols], sort=False, axis=1)
    
    final_df = pd.concat([final_df, new_full_row], sort=False, axis=0)
    
final_df = final_df[1:]

retrying after...1secs
retrying after...3secs
retrying after...2secs
retrying after...3secs
retrying after...2secs
retrying after...2secs
retrying after...3secs
retrying after...1secs
retrying after...2secs
retrying after...3secs
retrying after...3secs
retrying after...2secs
retrying after...3secs
retrying after...3secs
retrying after...2secs
retrying after...1secs
retrying after...1secs
retrying after...3secs
retrying after...3secs
retrying after...2secs
retrying after...2secs
retrying after...2secs
retrying after...2secs
retrying after...3secs
retrying after...2secs
retrying after...3secs
retrying after...2secs
retrying after...3secs
retrying after...1secs
retrying after...1secs
retrying after...2secs
retrying after...3secs
retrying after...2secs
retrying after...3secs
retrying after...2secs
retrying after...3secs
retrying after...3secs
retrying after...3secs
retrying after...2secs
retrying after...3secs
retrying after...1secs
retrying after...2secs
retrying after...3secs
retrying af

/Users/shaunmbarry/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.


Bad url: https://open.spotify.com/track/5yHk0TCVFH65aIY5stUUb3


SpotifyException: http status: 400, code:-1 - https://api.spotify.com/v1/tracks/5yHk0TCVFH65aIY5stUUb3:
 Only valid bearer authentication supported

In [194]:
final_df#[(final_df.shape[0]-1):final_df.shape[0]]

,Chart Artist Name,Spotify Artist Name,Chart Song Name,Spotify Song Name,Lev Dist,Predicted URL,Debut Year,Debut Date,Chart Appearances,Sp Artist Genres,Sp Album Genres,Sp Album Type,Sp Album Name
0,None,None,None,None,NaN,None,None,None,None,None,NaN,None,None
0,Backstreet Boys,Backstreet Boys,Show Me The Meaning Of Being Lonely,Show Me the Meaning of Being Lonely,0.0,https://open.spotify.com/track/3BsaRV5QIulYz2l...,2000,1/1/00,Hot100,boy band || dance pop || europop || pop,NaN,album,Millennium
0,Jennifer Day,Jennifer Day,The Fun Of Your Love,The Fun of Your Love,0.0,https://open.spotify.com/track/7oMsHuN8P6HFDgM...,2000,1/1/00,Country,deep contemporary country,NaN,album,The Fun of Your Love
0,Skip Ewing,Skip Ewing,Christmas Carol,Christmas Carol,0.0,https://open.spotify.com/track/7nyLB7rEsdfFwUp...,2000,1/1/00,Country,NaN,NaN,album,Following Yonder Star
0,Los Tucanes de Tijuana,Los Tucanes De Tijuana,Las Heladas,Las Heladas,0.0,https://open.spotify.com/track/50sTMaXwK2k8ZTg...,2000,1/1/00,Latin,banda || grupera || norteno || regional mexican,NaN,album,Amor Platónico
...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,Adele,Adele,Turning Tables,Turning Tables,0.0,https://open.spotify.com/track/5O6yk6U8L3rTbFs...,2011,5/7/11,Hot100,british soul || pop || uk pop,NaN,album,21
0,Beyonce,Beyoncé,Run The World (Girls),Run the World (Girls),1.0,https://open.spotify.com/track/1uXbwHHfgsXcUKf...,2011,5/7/11,"Hot100, RAB",dance pop || pop || post-teen pop || r&b,NaN,album,4
0,Intocable,Intocable,Prometi,Prometi,0.0,https://open.spotify.com/track/0V9Vqrn5w8f7ALF...,2011,5/7/11,Latin,banda || grupera || latin || latin pop || nort...,NaN,album,Intocable 2011
0,Mindless Behavior,Mindless Behavior,My Girl,My Girl,0.0,https://open.spotify.com/track/70jb2bIurVzYfxh...,2011,5/7/11,RAB,deep pop r&b || pop rap,NaN,album,#1 Girl


In [ ]:
final_df.to_csv('mp3_matching_y2k+.csv')

## Mess with audio features

In [89]:
analysis = spotify.audio_analysis('https://open.spotify.com/track/6SpLc7EXZIPpy0sVko0aoU?si=wWrTeaucREO1YqnGFwkc9A')

In [93]:
analysis.keys()

dict_keys(['meta', 'track', 'bars', 'beats', 'tatums', 'sections', 'segments'])

In [101]:
spotify.audio_features('https://open.spotify.com/track/6SpLc7EXZIPpy0sVko0aoU?si=wWrTeaucREO1YqnGFwkc9A')

[{'danceability': 0.517,
  'energy': 0.906,
  'key': 1,
  'loudness': -3.677,
  'mode': 1,
  'speechiness': 0.0735,
  'acousticness': 0.00272,
  'instrumentalness': 9.26e-06,
  'liveness': 0.113,
  'valence': 0.731,
  'tempo': 172.977,
  'type': 'audio_features',
  'id': '6SpLc7EXZIPpy0sVko0aoU',
  'uri': 'spotify:track:6SpLc7EXZIPpy0sVko0aoU',
  'track_href': 'https://api.spotify.com/v1/tracks/6SpLc7EXZIPpy0sVko0aoU',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/6SpLc7EXZIPpy0sVko0aoU',
  'duration_ms': 211520,
  'time_signature': 4}]

[]